# 01. Memory

In some applications like chatbots, it is important to remember previous interactions to keep the whole context of a conversation.
Memory does provide you an easy way to handle this.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import langchain as lc

import warnings
warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
# Chat message with no memory

history = lc.memory.ChatMessageHistory()
history.add_user_message("Hi")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='Hi', additional_kwargs={}, example=False),
 AIMessage(content='hello my friend!', additional_kwargs={}, example=False)]

In [5]:
# With stored memory

memory = lc.memory.ConversationBufferMemory()
memory.chat_memory.add_user_message("Hi")
memory.chat_memory.add_ai_message("Hello my friend!")
memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: Hello my friend!'}

In [7]:
llm = lc.OpenAI(temperature=0)
conversation = lc.chains.ConversationChain(
    llm=llm, memory=lc.memory.ConversationBufferMemory(),
    verbose=True)
conversation("Hi")




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi
AI:

> Finished chain.


{'input': 'Hi', 'history': '', 'response': ' Hi there! How can I help you?'}

In [8]:
conversation("I need to know the PM of India.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI:  Hi there! How can I help you?
Human: I need to know the PM of India.
AI:

> Finished chain.


{'input': 'I need to know the PM of India.',
 'history': 'Human: Hi\nAI:  Hi there! How can I help you?',
 'response': ' The Prime Minister of India is Narendra Modi. He has been in office since 2014.'}

In [9]:
conversation("So you maintaining the history?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI:  Hi there! How can I help you?
Human: I need to know the PM of India.
AI:  The Prime Minister of India is Narendra Modi. He has been in office since 2014.
Human: So you maintaining the history?
AI:

> Finished chain.


{'input': 'So you maintaining the history?',
 'history': 'Human: Hi\nAI:  Hi there! How can I help you?\nHuman: I need to know the PM of India.\nAI:  The Prime Minister of India is Narendra Modi. He has been in office since 2014.',
 'response': ' Yes, I am able to store and recall information from my context. Is there anything else I can help you with?'}

# 02. ConversationSummaryMemory

When inputs get long, we might not want to send the whole conversation, but rather a small summary.

In [10]:
!pip install tiktoken

In [11]:
review = "I ordered Pizza Salami for 9.99$ and it was awesome! \
The pizza was delivered on time and was still hot when I received it. \
The crust was thin and crispy, and the toppings were fresh and flavorful. \
The Salami was well-cooked and complemented the cheese perfectly. \
The price was reasonable and I believe I got my money's worth. \
Overall, I am very satisfied with my order and I would recommend this pizza place to others."

memory = lc.memory.ConversationSummaryBufferMemory(llm=llm, 
                                                   max_token_limit=100)
memory.save_context(
    {"input": "Hello, how can I help you today?"},
    {"output": "Could you analyze a review for me?"},
)
memory.save_context(
    {"input": "Sure, I'd be happy to. Could you provide the review?"},
    {"output": f"{review}"},
)

In [12]:
conversation = lc.chains.ConversationChain(llm=llm, memory=memory, verbose=True)

In [13]:
conversation("Thank you very much!")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the review.
AI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.
Human: Thank you very much!
AI:

> Finished chain.


{'input': 'Thank you very much!',
 'history': "System: \nThe human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the review.\nAI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.",
 'response': " You're welcome! It's always a pleasure to share my experiences. Is there anything else I can help you with?"}

In [14]:
memory.load_memory_variables({})

{'history': "System: \n\nThe human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the review, which was a positive review of a pizza order with Salami for 9.99$. The pizza was delivered on time and was still hot, with a thin and crispy crust and fresh and flavorful toppings. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and the customer was very satisfied with their order, recommending the pizza place to others.\nHuman: Thank you very much!\nAI:  You're welcome! It's always a pleasure to share my experiences. Is there anything else I can help you with?"}